# Statistical efficiency of estimators

This notebook computes the RMSE between ground-truth parameters and various estimators, using an experiment based on synthetically-generated data.
It basically follows the experiment used to produce Figure 1 in Hajek et al., *Minimax-optimal Inference from Partial Rankings* (2014).

In [1]:
import choix
import math
import matplotlib.pyplot as plt
import numpy as np
import pickle

np.random.seed(24)

Helper functions.

In [2]:
def indep_break(full_rankings, size):
    """Break full rankings into independent & smaller (partial) ones."""
    data = list()
    for r in full_rankings:
        perm = np.random.permutation(n_items)
        for i in range(0, n_items, size):
            idx = np.sort(perm[i:i+size])
            data.append(r[idx])
    return data
            
def full_break(rankings):
    """Break (partial) rankings into all pairwise comparisons."""
    comparisons = list()
    for ranking in rankings:
        for i, winner in enumerate(ranking):
            for loser in ranking[i+1:]:
                comparisons.append((winner, loser))
    return comparisons

Experimental settings.

In [3]:
interval = 4.0  # Dynamic range of the parameters
log_n_items = 10  # Base-2 logarithm of number of items
n_rankings = 64  # Number of full rankings generated.

n_items = 2**log_n_items
params = choix.generate_params(n_items, interval=interval)
rankings = choix.generate_rankings(params, n_rankings, size=n_items)

sizes = np.logspace(1, log_n_items, num=log_n_items, endpoint=True, base=2, dtype=int)

`res` will contain the experimental results.

In [4]:
res = {
    "sizes": sizes,
    "LSR": list(),
    "ML": list(),
    "GMM-F": list(),
    "ML-F": list(),
    "CRLB": list(),
    "OLB": 0.0,
    "XLB": 0.0,
}

## Empirical efficiency of estimators

In [5]:
%%time
for size in sizes:
    print(".", end="", flush=True)
    rk_data = indep_break(rankings, size)
    # LSR.
    est = choix.lsr_rankings(n_items, rk_data)
    res["LSR"].append(choix.rmse(params, est))
    # Maximum-likelihood.
    est = choix.ilsr_rankings(n_items, rk_data)
    res["ML"].append(choix.rmse(params, est))

    pw_data = full_break(rk_data)
    # GMM full-breaking.
    est = choix.lsr_pairwise(n_items, pw_data)
    res["GMM-F"].append(choix.rmse(params, est))
    # Maximum-likelihood full-breaking.
    est = choix.ilsr_pairwise(n_items, pw_data)
    res["ML-F"].append(choix.rmse(params, est))
print()

..........
CPU times: user 15min 2s, sys: 5.21 s, total: 15min 7s
Wall time: 15min


## Oracle lower bound

$$
n \cdot \text{MSE}
    = \inf_{\hat{\theta}} \sup_{\theta^\star \in \Theta_b} E[\lVert \hat{\theta} - \theta^\star \rVert_2^2]
    \ge \frac{1}{2 I(\mu) + \frac{2 \pi^2}{b^2 (d_1 + d_2)}} \frac{(n-1)^2}{mk}
$$

In our case:

- $I(\mu) = 1$ (given in the paper)
- $n = n_{\text{items}}$
- $mk = n_{\text{rankings}} \cdot n_{\text{items}}$
- $d_1 = d_2 = n_{\text{rankings}}$
- $b = \text{interval} / 2$

and hence,

$$
\text{MSE} =
    \frac{1}{2 + \frac{4 \pi}{\text{interval}^2 n_{\text{rankings}}}}
    \frac{(n_{\text{items}} - 1)^2}{n_{\text{items}}^2 n_{\text{rankings}}}
$$

In [6]:
a = 1 / (2 + 4 * math.pi / (interval**2 * n_rankings))
b = (n_items - 1)**2 / (n_items**2 * n_rankings)
res["OLB"] = np.sqrt(a * b)

## Cramér-Rao lower bound

This lower bound is only valid for *unbiased* estimators.

$$
n \cdot \text{MSE}
    = \inf_{\hat{\theta} \in \mathcal{U}} \sup_{\theta^\star \in \Theta_b} E[\lVert \hat{\theta} - \theta^\star \rVert_2^2]
    \le \left( 1 - \frac{1}{k} \sum_{\ell=1}^k \frac{1}{\ell} \right)^{-1} \frac{(n-1)^2}{mk}
$$

In [7]:
for size in sizes:
    H_k = sum(1.0 / ell for ell in range(1, size+1))
    a = 1 / (1 - H_k / size)
    crlb = np.sqrt(a * (n_items - 1)**2 / (n_items**2 * n_rankings))
    res["CRLB"].append(crlb)

We can furthermore simplify the Cramér-Rao lower-bound as follows.

$$
\left( 1 - \frac{1}{k} \sum_{\ell=1}^k \frac{1}{\ell} \right)^{-1} \frac{(n-1)^2}{mk}
    \le \frac{(n-1)^2}{mk}
$$

In [8]:
res["XLB"] = np.sqrt((n_items - 1)**2 / (n_items**2 * n_rankings))

## Saving the data

In [9]:
with open("data/statistical-efficiency.pickle", "wb") as f:
    pickle.dump(res, f)